In [92]:
import numpy as np
from PIL import Image
import time, re, os, sys
import matplotlib.pyplot as plt
import cv2
from scipy.spatial.distance import pdist,squareform
from array2gif import write_gif
#from util import *


In [93]:
FOLDER = ''
POINT_NUM = 10000
LENGTH = 100
CLUSTER_NUM = 4
GIF_path = './GIF'
colormap= np.random.choice(range(256),size=(100,3))


In [94]:
def openImage(path):
	image = cv2.imread(path)
	H, W, C = image.shape
	image_flat = np.zeros((W * H, C))
	for h in range(H):
		image_flat[h * W:(h + 1) * W] = image[h]
	return image_flat,H,W

In [95]:
def precomputed_kernel(X, gamma_s, gamma_c):
    n=len(X)
    # S(x) spacial information
    S=np.zeros((n,2))
    for i in range(n):
        S[i]=[i//100,i%100]
    print(pdist(S,'sqeuclidean').shape)
    K=squareform(np.exp(-gamma_s*pdist(S,'sqeuclidean')))*squareform(np.exp(-gamma_c*pdist(X,'sqeuclidean')))
    print(K.shape)
    return K

In [96]:
def visualize(C,k,H,W):
    '''
    @param C: (10000) belonging classes ndarray
    @param k: #clusters
    @param H: image_H
    @param W: image_W
    @return : (H,W,3) ndarray
    '''
    colors= colormap[:k,:]
    res=np.zeros((H,W,3))
    for h in range(H):
        for w in range(W):
            #np.array(res)[h,w,:] = np.array(colors)[C[h*W+w]]
            res[h,w,:] = colors[C[h*W+w]]

    return res.astype(np.uint8)

In [97]:
def kmeans(CLUSTER_NUM, Gram, H, W, init=None):
    Cluster = np.zeros((CLUSTER_NUM, Gram.shape[1]))
    if init == 'random':
        random_pick=np.random.randint(low=0,high=Gram.shape[0],size=CLUSTER_NUM)
        Cluster = Gram[random_pick,:]
    if init == 'gaussian':
        X_mean=np.mean(Gram,axis=0)
        X_std=np.std(Gram,axis=0)
        for c in range(Gram.shape[1]):
            Cluster[:,c]=np.random.normal(X_mean[c],X_std[c],size=CLUSTER_NUM)
      

    # kmeans++ init
    
    Cluster[0]=Gram[np.random.randint(low=0,high=Gram.shape[0],size=1),:]
    for c in range(1,CLUSTER_NUM):
            Dist=np.zeros((len(Gram),c))
            for i in range(len(Gram)):
                for j in range(c):
                    Dist[i,j]=np.sqrt(np.sum((Gram[i]-Cluster[j])**2))
            Dist_min=np.min(Dist,axis=1)
            sum=np.sum(Dist_min)*np.random.rand()
            for i in range(len(Gram)):
                sum-=Dist_min[i]
                if sum<=0:
                    Cluster[c]=Gram[i]
                    break
    # kmeans++
    diff = 1e9
    eps = 1e-9
    count = 1
    # Classes of each Xi
    C=np.zeros(len(Gram),dtype=np.uint8)
    segments=[]
    while diff > eps:
        # E-step
        for i in range(len(Gram)):
            dist=[]
            for j in range(CLUSTER_NUM):
                dist.append(np.sqrt(np.sum((Gram[i]-Cluster[j])**2)))
            C[i]=np.argmin(dist)
        
        #M-step
        New_Mean=np.zeros(Cluster.shape)
        for i in range(CLUSTER_NUM):
            belong=np.argwhere(C==i).reshape(-1)
            for j in belong:
                New_Mean[i]=New_Mean[i]+Gram[j]
            if len(belong)>0:
                New_Mean[i]=New_Mean[i]/len(belong)

        diff = np.sum((New_Mean - Cluster)**2)
        Cluster=New_Mean
        # visualize
        segment = visualize(C, CLUSTER_NUM, H, W)
        segments.append(segment)
        print('iteration {}'.format(count))
        for i in range(CLUSTER_NUM):
            print('k={}: {}'.format(i + 1, np.count_nonzero(C == i)))
        print('diff {}'.format(diff))
        print('-------------------')
        cv2.imshow('', segment)
        cv2.waitKey(1)
    return C, segments

In [98]:
def kernelkmeans(path):
    image_flat, H, W = openImage(path)
    gamma_s = 0.001
    gamma_c = 0.001
    Gram = precomputed_kernel(image_flat, gamma_s, gamma_c)
    C, segments = kmeans(CLUSTER_NUM, Gram, H, W, init='gaussian')
    # save_gif
    for i in range(len(segments)):
        segments[i] = segments[i].transpose(1, 0, 2)
    filename = path.split('.')[0] + '.gif'
    gif_path = os.path.join(GIF_path, filename)
    write_gif(segments, gif_path, fps=2)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

        




In [99]:
kernelkmeans('image1.png')


(49995000,)
(10000, 10000)
iteration 1
k=1: 744
k=2: 1744
k=3: 7512
diff 211.72958996086163
-------------------
iteration 1
k=1: 781
k=2: 2188
k=3: 7031
diff 5.276690652270715
-------------------
iteration 1
k=1: 791
k=2: 2393
k=3: 6816
diff 0.8012259586438704
-------------------
iteration 1
k=1: 794
k=2: 2493
k=3: 6713
diff 0.1432177691792697
-------------------
iteration 1
k=1: 795
k=2: 2528
k=3: 6677
diff 0.013459743266279123
-------------------
iteration 1
k=1: 796
k=2: 2541
k=3: 6663
diff 0.0021938435851602375
-------------------
iteration 1
k=1: 796
k=2: 2544
k=3: 6660
diff 0.00011546162421341757
-------------------
iteration 1
k=1: 796
k=2: 2544
k=3: 6660
diff 0.0
-------------------


In [100]:
kernelkmeans('image2.png')

(49995000,)
(10000, 10000)
iteration 1
k=1: 7539
k=2: 871
k=3: 1590
diff 200.29600203988915
-------------------
iteration 1
k=1: 7470
k=2: 880
k=3: 1650
diff 0.40625434070176425
-------------------
iteration 1
k=1: 7462
k=2: 882
k=3: 1656
diff 0.0061218474466249555
-------------------
iteration 1
k=1: 7463
k=2: 882
k=3: 1655
diff 7.778694298067207e-05
-------------------
iteration 1
k=1: 7463
k=2: 882
k=3: 1655
diff 0.0
-------------------
